# Overfitting, Test-Train Splitting, and Cross-Validation
This notebook will discuss overfitting data and cross-validation using the [``numpy``](http://www.numpy.org/), [``matplotlib``](https://matplotlib.org/), and [``scikit-learn``](https://scikit-learn.org/stable/) libraries.

``numpy`` is a scientific computing library for Python. It is "lower level" than other libraries that we'll be using later in this course (in this context "lower level" means "has more nitty-gritty details"). Most of the data science libraries for Python are built on top of ``numpy``.

``matplotlib`` is a visualization library for Python. Its name comes from the fact that was originally based on the plotting functionality of Matlab. Most of the visualization libraries that one will come across in Python are built upon ``matplotlib``.

``scikit-learn`` is a library that has many machine learning algorithms implemented. We will be using ``scikit-learn`` more in the next few examples, but for now we're just using it to split our data.

In [1]:
# Most Python code uses the abbreviation np for numpy
import numpy as np
# Most python code uses the abbreviation plt for matplotlib's pyplot module
import matplotlib.pyplot as plt
# We'll be just using a couple of functions from sklearn's model_selection
# module, so don't bother with an abbreviation.
import sklearn.model_selection

ModuleNotFoundError: No module named 'sklearn'

# Our goal

Our goal will be to find the best model to fit our data without over-fitting.

The class of models that we'll be considering is all polynomials, so, a linear fit (1st degree polynomial), a quadratic fit (2nd degree polynomial), a cubic fit (3rd degree polynomial), etc.

To rephrase the above, our goal is to find the number $P$ such that a $P$th degree polynomial is the best model for fitting our data (we have been referring to this number as $\lambda$).

The parameters for any given polynomial fit are the coefficients of that polynomial (so, for a line $y=\beta x+\alpha$, the parameters are $\beta$ and $\alpha$). Sometimes this parameter for the model $P$ is called a "hyperparameter", to distinguish it from the parameters of the model.

# Generating our Data
In this example, we'll generate some random data that we'll be using to work on. First, let's create a variable ``n`` for the number of data points we'll have

In [2]:
n = 30

While we want our data to be random, for our purposes in this example we will want the data that we get to be repeatable (otherwise we could never have slides that use the same data as our notebook!). To do this, we set something called a random seed using [``np.random.seed``](https://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.random.seed.html). This ensures that when we generate random data using ``np.random``, we will always get the same thing. Here we use 7 as an arbitrary choice (not really, it's my favorite number).

In [3]:
np.random.seed(7)

If we want to start getting truly (or, less predictable) random numbers again, then we would just call ``np.random.seed()`` with no parameters.

## The x values

Let's define ``x`` to be a set of points selected uniformly at random from the interval $[0, 3.14)$, using the [``np.random.uniform``](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.uniform.html) function.

In [4]:
x = np.random.uniform(0,3.14,n)
x

array([0.23960803, 2.44894501, 1.37660499, 2.27168066, 3.07088707,
       1.69087703, 1.57351826, 0.22624056, 0.8428984 , 1.56963105,
       2.13278219, 2.52374057, 1.19615516, 0.20704013, 0.90477718,
       2.85612368, 0.67003001, 1.41966924, 2.9239869 , 0.07818357,
       1.8857236 , 2.98340663, 0.72315104, 1.72225835, 2.8546631 ,
       0.41815206, 1.6435155 , 2.35628696, 2.10070158, 1.46874398])

Note that the first number in the series should be 0.23960803.

## The y values
Let's define ``y`` using the function $y=\sin(x)$ with some normally distributed noise added. The normally distributed noise is generated using the [``np.random.normal``](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.normal.html) function. We'll have the noise's distribution be centered at $0$ and have a standard deviation of $0.1$.

In [ ]:
np.random.seed(1)
y = np.sin(x)+np.random.normal(0, 0.1, n)
#y = np.sin(x)

y

Note that the first number should be 0.39975641.

## Plotting the data
Let's create a scatterplot of the data using the [``plt.scatter``](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.scatter.html) function.

One thing to note about ``matplotlib`` is that each call to ``plt`` will draw additional things on to your "current" plot. Only once you have drawn everything you need should you call [``plt.show``](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.show.html), which will make the plot appear on-screen.

In [ ]:
plt.scatter(x, y)
plt.show()

# Fitting Various Models to Our Data

## Linear regression

In this section we will fit the data to a line. In future examples we'll use more sophisticated regressions packages from ``statsmodels``, but for now we will just use the [``np.polyfit``](https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html) function.

### Computing the linear fit

When we call ``np.polyfit`` with our data, it will return a ``np.array`` with the polynomial coefficients, which we convert to a [``np.poly1d``](https://docs.scipy.org/doc/numpy/reference/generated/numpy.poly1d.html), which is ``numpy``'s polynomial class.

In [ ]:
linear_fit_coefficients = np.polyfit(x, y, deg=1)
print(linear_fit_coefficients)

In [ ]:
linear_fit = np.poly1d(linear_fit_coefficients)
print('y = ')
print(linear_fit)

### Computing RMSE

We'll be using the function [``np.polyval``](https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html) to evaluate our polynomial. Let's see what the ``y_predicted`` or $\hat y$ value would be.

In [ ]:
y_predicted = np.polyval(linear_fit, x)
y_predicted

We can compute the residual for each sample by just subtracting one from the other.

In [ ]:
linear_fit_residual = y - y_predicted
linear_fit_residual

Let's compute the RMSE, since that will give us a single value for our fit.

We'll compute the "root" part of RMSE using ``np.sqrt``, the "mean" part of RMSE using ``np.mean``, and the "square" part of RMSE using ``np.square`` as follows.

In [ ]:
linear_fit_rmse = np.sqrt(np.mean(np.square(linear_fit_residual)))
print("Linear fit RMSE = " + str(linear_fit_rmse))

### Plotting the fit

A more intuitive way to look at a fit would be to plot it next to the data that we want to view.

To do this, we will need to create data for the $x$ values line. Let's call this ``x_line``, and use [``np.arange``](https://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html) to set it to be taking 200 samples from ``x.min()`` to ``x.max()``.

In [ ]:
x_line_step = (x.max() - x.min()) / 200
x_line = np.arange(x.min(), x.max(), x_line_step)
display(x_line)

Now we can plot the our data and the prediction together. We'll use ``np.polyval`` to evaluate ``linear_fit`` at ``x_line``.

In [ ]:
plt.scatter(x, y, label='Data')
plt.plot(x_line, np.polyval(linear_fit, x_line), label='Linear Fit')
plt.legend()
plt.show()

Do you feel like this fit captures the underlying trend of the data? 

## Coding Dojo: Quadratic fit

The linear fit doesn't fit our data very well. So maybe we can do better if we try to fit a slightly more complicated model -- a quadratic polynomial. We can do this again with ``np.polyfit``, with the only difference being that we'll use ``deg=2``.

That's great! If going from $x$ to $x^2$ was good, then maybe $x^3$ is better! Maybe $x^4$ is better!

### Creating a function

We'll be running the same code over and over and over again, so it'll be convenient to write a function to do it for us. Let's write the function ``FitPolynomial``, which will have ``P`` be the degree of our polynomial to fit.

In [ ]:
def FitPolynomial(P):
  # Compute and print the polynomial fit!
  degree_P_fit = np.poly1d(np.polyfit(x, y, deg=P))
  print('y = ')
  print(degree_P_fit)
  # Compute and print the RMSE
  degree_P_fit_residual = y - np.polyval(degree_P_fit, x)
  degree_P_fit_rmse = np.sqrt(np.mean(np.square(degree_P_fit_residual)))
  print(str(P) + 'th degree fit RMSE = ' + str(degree_P_fit_rmse))
  # Plot the fit
  plt.plot(x_line, np.polyval(degree_P_fit, x_line), label=str(P) + 'th Degree Fit')
  plt.scatter(x, y, label='Data')
  # Use the function plt.ylim to ensure that we see the same y axis for every fit
  # (we'll see why shortly)
  plt.ylim([0, 1.2])
  plt.legend()
  plt.show()

## Cubic fit

In [ ]:
FitPolynomial(3)

## 4th degree fit

In [ ]:
FitPolynomial(4)

## 6th degree fit

In [ ]:
FitPolynomial(6)

## 9th degree fit

In [ ]:
FitPolynomial(9)

## 12th degree fit

In [ ]:
FitPolynomial(12)

What about this fit? Does it still look good to you?

## 16th degree fit

In [ ]:
FitPolynomial(16)

## Finding the Best Hyperparameter

But our RMSE has kept going down!

Let's plot polynomial degree on the X axis versus RMSE on the Y axis all the way through from degree 0 (a constant) through a degree 16 polynomial.

In [ ]:
# This is going to be our X data. Let's call it degrees and set it to all of
# the degrees we will check, from 0 through 16.
degrees = np.arange(17)

# This is going to be our Y data. It will be the RMSE for each degree. We don't
# know the values yet (we will compute them), so let's initialize it to zero.
rmse_for_degree = np.zeros(17)

# And for each degree, let's compute the RMSE
for P in degrees:
  degree_P_fit = np.poly1d(np.polyfit(x, y, deg=P))
  residual = y - np.polyval(degree_P_fit, x)
  rmse = np.sqrt(np.mean(np.square(residual)))
  # Save the RMSE in our Y data
  rmse_for_degree[P] = rmse

# And let's plot the results.
plt.plot(degrees, rmse_for_degree)
plt.xlabel('Polynomial degree')
plt.ylabel('RMSE')
plt.show()

Our original goal was to find the best hyperparameter (the polynomial degree that is the best fit for our data), but it's not clear from this graph what that is!

# Training/Test Split

In this section, we'll split our data into two sets -- one for testing and one for training. We'll tune our hyperparameter by evaluating how a model trained on the training set performs on the testing set.

## Splitting the data

Here we produce a set of indices for training and testing. The indices are numbers from ``0`` to  ``n`` that indicate positions in the ``x`` and ``y`` arrays.

We do this by using the [``sklearn.model_selection.train_test_split``](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function.

We'll split the indices evenly using the ``test_size`` parameter. The ``random_state`` parameter sets the random seed for the operation.

In [ ]:
all_indices = np.arange(n)
print('All indices:')
print(all_indices)

In [ ]:
train_indices, test_indices = sklearn.model_selection.train_test_split(
    all_indices, test_size=0.5, shuffle=True, random_state=1)
train_indices = np.sort(train_indices)
test_indices = np.sort(test_indices)
print('Training set indices:')
print(train_indices)
# print(np.sort(train))
print('Test set indices:')
print(test_indices)

In [ ]:
x_train = x[train_indices]
y_train = y[train_indices]
x_test  = x[test_indices]
y_test  = y[test_indices]

print('x = ')
print(x)
print('x_train = ')
print(x_train)

Let's visualize the split by plotting the training and the testing data in different colors.

In [ ]:
plt.scatter(x_train, y_train, label='Train', c='b')
plt.scatter(x_test, y_test, label='Test', c='r')
plt.legend()
plt.show()

## Linear fit

Now let's compute a linear (``deg=1``) fit on the training data, and look at its plot.

In [ ]:
# Compute our linear fit 
linear_fit = np.poly1d(np.polyfit(x_train, y_train, deg=1))
# Plot the fit
plt.scatter(x_train, y_train, label='Train', c='b')
plt.scatter(x_test,  y_test,  label='Test', c='r')
plt.plot(x_line, np.polyval(linear_fit, x_line), label='Linear Fit', c='b')
plt.legend()
plt.show()

Let's compute the RMSE for this model on the training data and the testing data.

In [ ]:
train_residual = y_train - np.polyval(linear_fit, x_train)
train_rmse = np.sqrt(np.mean(np.square(train_residual)))
print("RMSE training data = " + str(train_rmse))
test_residual = y_test - np.polyval(linear_fit, x_test)
test_rmse = np.sqrt(np.mean(np.square(test_residual)))
print("RMSE testing data = " + str(test_rmse))

## Creating a function

As before, we're going to take a look at how this performs for many different polynomial degrees. Let's create a function to do the repetetive work for us.

In [ ]:
def FitPolynomailToTrainTestSplit(P):
  # Compute our polynomial fit
  degree_P_fit = np.poly1d(np.polyfit(x_train, y_train, deg=P))
  
  # Plot the fit
  plt.scatter(x_train, y_train, label='Train', c='b')
  plt.scatter(x_test, y_test, label='Test', c='r')
  plt.plot(x_line, np.polyval(degree_P_fit, x_line),
           label='Degree ' + str(P) + ' Fit', c='b')
  plt.legend()
  plt.show()
  
  # Compute and print the RMSE
  train_residual = y_train - np.polyval(degree_P_fit, x_train)
  train_rmse = np.sqrt(np.mean(np.square(train_residual)))
  print("RMSE training data = " + str(train_rmse))
  test_residual = y_test - np.polyval(degree_P_fit, x_test)
  test_rmse = np.sqrt(np.mean(np.square(test_residual)))
  print("RMSE testing data = " + str(test_rmse))

## Quadratic fit

Let's run the same code as before, now with ``deg=2``.

In [ ]:
FitPolynomailToTrainTestSplit(2)

Indeed the RMSE is going down like before, for both the training and testing sets.

## Coding Dojo: Run the 6th polynomial fit! 

## 8th degree fit

Let's do the same thing, now with a 8th degree polynomial.

In [ ]:
FitPolynomailToTrainTestSplit(8)

Something interesting has happened here! Our training data's error is going down, but our testing data has spiked up!

And if we look at our plot, the line fits the blue dots well, but is missing the red dots by a lot!

## Finding the Best Hyperparameter (again!)

We want to know the relationship between RMSE on the *test* set versus the polynomial degree.

Let's plot that for all polynomial degrees from 0 to 13 (along with the RMSE for the training error).

In [ ]:
# This is going to be our X data. Like last time, it's the degrees we'll test,
# this time 0 through 12.
degrees = np.arange(13)

# This is going to be our Y data. We have two plots now, the RMSE for training
# data and RMSE for test data.
train_rmse_for_degree = np.zeros(len(degrees))
test_rmse_for_degree = np.zeros(len(degrees))

# And for each degree, let's compute the RMSE
for P in degrees:
  # Compute our fit
  degree_P_fit = np.poly1d(np.polyfit(x_train, y_train, deg=P))

  # Compute our residual and RMSE for training and testing
  train_residual = y_train - np.polyval(degree_P_fit, x_train)
  train_rmse = np.sqrt(np.mean(np.square(train_residual)))
  test_residual = y_test - np.polyval(degree_P_fit, x_test)
  test_rmse = np.sqrt(np.mean(np.square(test_residual)))
  
  # Save the RMSE in our Y axis data
  train_rmse_for_degree[P] = train_rmse
  test_rmse_for_degree[P] = test_rmse

  # And let's plot the results.
plt.plot(degrees, train_rmse_for_degree, c='b', label='Train')
plt.plot(degrees, test_rmse_for_degree, c='r', label='Test')
plt.xlabel('Polynomial degree')
plt.ylabel('RMSE')
plt.ylim([0, 1])
plt.legend()
plt.show()

This gives us a clearer picture of what range of hyperparameters may be best for this data.

# k-Fold Cross-Validation

In the above example, we dividied our data at random into 2 sets -- one for training and one for testing. Now we know that the correct way to approach model evalutation is to include a "validation" set as well. 

Let's see how that is done in the context of cross validation.

For cross-validation, the training data is divided into folds, each of which is treated as its own validation data set, using all of the other training data as its training data set.

We can use the [``sklearn.model_selection.KFold``](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) function to generate mutliple training and validation sets.

In this example, let's do 4-Fold Cross-Validation

## Visualizing the split

Let's print out the 4 different sets of test and training indices that we'll use. For each fold, the training indices are everything not being used for validation.

In [ ]:
# kf_xval is what we will use to iterate over our splits
n_folds =4
kf_xval = sklearn.model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=7)

# We'll want to label each fold as 0, 1, 2, 3 on the graph and when we print
# the data. We'll use the fold_index variable for that.
fold_number = 0
for fold_train_indices, fold_valid_indices in kf_xval.split(x_train):
  # fold_train_indices, fold_valid_indices are the indices from 0 through 15
  # (the size of x_train). Print them.
  print('Fold ' + str(fold_number) + ' training indices: ')
  print(fold_train_indices)
  print('Fold ' + str(fold_number) + ' validation indices: ')
  print(fold_valid_indices)
  print('')

  # Use them to sub-divide x_train and y_train into the training and validation
  # sets for this fold.
  x_fold_train = x_train[fold_train_indices]
  y_fold_train = y_train[fold_train_indices]
  x_fold_valid = x_train[fold_valid_indices]
  y_fold_valid = y_train[fold_valid_indices]

  # Plot the fold's valiation data points
  plt.scatter(x_fold_valid, y_fold_valid, label='Fold ' + str(fold_number))
  
  # Increment our fold index.
  fold_number += 1

# Show the plot
plt.legend()
plt.show()

## Finding the best Hyperparameter (again again!)

In the code below, we try all of our different degree fits on the above sets, and we plot the resulting error for each.

Our X axis is polynomial degree. On the Y axis we'll have the mean training fit and the mean validation fit.

In [ ]:
# This is going to be our X data. Like last time, it's the degrees we'll test.
degrees = np.arange(11)

# This is going to be our Y data. We have two plots now, the RMSE for training
# data and RMSE for test data. Because we need to take the mean of the RMSE for
# all folds, we'll create a matrix with entries for each degree and each fold.
train_rmse_for_degree = np.zeros([len(degrees), n_folds])
valid_rmse_for_degree = np.zeros([len(degrees), n_folds])

# We'll use fold_number to keep track of which fold we're on, so we know which
# entry to save it in.
fold_number = 0

# Iterate over every training/validation split
for fold_train_indices, fold_valid_indices in kf_xval.split(x_train):
  # fold_train_indices and fold_valid_indices are the indices. Use them to
  # sub-divide x_train and y_train into the training and validation sets for
  # this fold.
  x_fold_train = x_train[fold_train_indices]
  y_fold_train = y_train[fold_train_indices]
  x_fold_valid = x_train[fold_valid_indices]
  y_fold_valid = y_train[fold_valid_indices]

  # Iterate over every polynomial degree P (this is now exactly like before)
  for P in degrees:
    # Fit the degree P polynomial using the fold's training data
    degree_P_fit = np.poly1d(np.polyfit(x_fold_train, y_fold_train, deg=P))

    # Compute the residual and RMSE for training and validation
    train_residual = y_fold_train - np.polyval(degree_P_fit, x_fold_train)
    valid_residual = y_fold_valid - np.polyval(degree_P_fit, x_fold_valid)
    train_rmse = np.sqrt(np.mean(np.square(train_residual)))
    valid_rmse = np.sqrt(np.mean(np.square(valid_residual)))
    
    # Save the RMSE
    train_rmse_for_degree[P, fold_number] = train_rmse
    valid_rmse_for_degree[P, fold_number] = valid_rmse

  # Increment the index for the fold we're on.
  fold_number += 1

# Now let's get the mean training and validation RMSE across all folds
mean_train_rmse_for_degree = np.mean(train_rmse_for_degree,1)
mean_valid_rmse_for_degree = np.mean(valid_rmse_for_degree,1)

# print('Training RMSE matrix. Folds are horizontal, degree is vertical')
# print(train_rmse_for_degree)
# print('Mean training RMSE for each degree.')
# print(mean_train_rmse_for_degree)
  
# And let's plot them!
plt.plot(degrees, mean_train_rmse_for_degree, label='Mean Training RMSE')
plt.plot(degrees, mean_valid_rmse_for_degree, label='Mean Validation RMSE')
plt.xlabel('Polynomial degree')
plt.ylabel('RMSE')
plt.ylim([0, 0.5])
plt.legend()
plt.show()

## Evaluating the model on the test data

From the above plot, a degree $P=2$ fit is the best hyperparameter selection. Let's now train our model on all training data using that hyperparameter, and evaluate it on the test set.



In [ ]:
best_P = 2
best_P_fit = np.poly1d(np.polyfit(x_train, y_train, deg=best_P))

best_P_test_residual = y_test - np.polyval(best_P_fit, x_test)
best_P_test_rmse = np.sqrt(np.mean(np.square(best_P_test_residual)))

print('P=2 Test RMSE = ' + str(best_P_test_rmse))

In [ ]:
# And let's everything together!
plt.plot(degrees, mean_train_rmse_for_degree, label='Mean Training RMSE')
plt.plot(degrees, mean_valid_rmse_for_degree, label='Mean Validation RMSE')
plt.scatter([best_P], [best_P_test_rmse], label='Test RMSE', c='r')
plt.xlabel('Polynomial degree')
plt.ylabel('RMSE')
plt.ylim([0, 0.5])
plt.legend()
plt.show()

# Coding Dojo: Leave-one-out cross validation

Try modifying the $k-$fold cross validation example above to use a [``LeaveOneOut``](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html) cross-validator instead.

In [ ]:
# We used kf_xval before. Let's use xf_loocv
loo_xval = sklearn.model_selection.LeaveOneOut()

# the number of folds we'll have now is the length of the training data.
n_folds = len(x_train)

# Copy-paste what we did for k-fold cross validation here, now using loo_xval
# instead of kf_xval. How does it compare?